In [48]:
import pdfplumber
import numpy as np
from glob2 import glob
import pandas as pd
import re

pd.set_option('max_columns',100)

In [4]:
file_list_ali = glob('**/*淘宝*.pdf') + glob('**/*阿里*.pdf')

file_list_jd = glob('**/*京东*.pdf')n n  n mmmmn m m

file_list_gh=glob('**/*工商*.pdf')+glob('**/*工行*.pdf')

In [5]:
len(file_list_ali),len(file_list_gh),len(file_list_jd)

(349, 363, 363)

In [6]:
file_list_jd[6]

'Foreclosure_Reports\\【一拍】北京市丰台区光彩路72号院6号楼1层1单元101（187.84㎡）\\京东网询评估报告.pdf'

In [34]:
price_table_jd=pd.DataFrame()
failed_jd_files = []
for filename in file_list_jd:
    try:
        pdf_content=pdfplumber.open(filename)
        panel_price=pdf_content.pages[1].extract_table()
        panel_price=pd.DataFrame(np.array(
            panel_price)[:,1],index=np.array(panel_price)[:,0]).T
        list_a=pdf_content.pages[2].extract_text().split('\n')
        price_date=list(filter(
            lambda x: '本次网络询价时间为' in x,list_a))[0].split('：')[1]
        panel_price['price_date']=price_date
        price_table_jd=pd.concat([price_table_jd,panel_price],axis='rows')
    except:
        print('failed:',filename)  # excellent usage of try... except
        failed_jd_files.append(filename)
        
#price_table_jd['price_date']=pd.to_datetime(
#    price_table_jd['price_date'],format='%Y年%m月%d日')
#price_table_jd[['bedrooms','livingrooms','restrooms'
#               ]]=price_table_jd['户型'].str.split('[室厅卫]', expand=True)[[0,1,2]]
#price_table_jd=price_table_jd.reset_index(drop=True)    

failed: Foreclosure_Reports\【一拍】北京市东城区北吉祥胡同6号2幢1层、3幢1层房屋\京东网络询价.pdf
failed: Foreclosure_Reports\【一拍】北京市朝阳区和敬路4号院15号楼2层2单元201（74.06㎡）\京东评估报告.pdf
failed: Foreclosure_Reports\【一拍】北京市朝阳区小庄6号3号楼25至26层29D（370.58㎡）\评估报告-京东.pdf
failed: Foreclosure_Reports\【一拍】北京市朝阳区望京东园六区604号楼13层3单元1301\评估报告-北京市朝阳区望京东园六区604号楼13层3单元1301号.pdf
failed: Foreclosure_Reports\【一拍】北京市门头沟区绿岛家园2号楼2层5单元201号房屋\绿岛家园2号楼2层5单元201号京东报告.pdf
failed: Foreclosure_Reports\【二拍】丰台区富锦嘉园四区6号楼1-601号\京东报告.pdf
failed: Foreclosure_Reports\【二拍】北京市门头沟区惠民家园房产一套\京东.pdf


In [8]:
price_table_ali=pd.DataFrame()
for filename in file_list_ali:
    try:
        pdf_content=pdfplumber.open(filename)
        panel_price=pdf_content.pages[0].extract_text().split('\n')

        search_list=['房产名称','房屋类型','房产坐落',
                     '所在小区','建筑面积','建筑面积',
                     '户型','朝向','所在楼层',
                     '建成年代','规划用途','单位面积价格','财产参考总价','本次网络询价时点为']
        info_dict={}
        for name in search_list:
            for x in panel_price:
                if name in x:
                    info_dict[name]=x.replace(name,'').replace(' ','').replace('：','')
        panel_price=pd.DataFrame(info_dict.values(),index=info_dict.keys()).T
        price_table_ali=pd.concat([price_table_ali,panel_price],axis='rows')
    except:
        print('failed:',filename)
price_table_ali=price_table_ali.reset_index(drop=True)
price_table_ali=price_table_ali.dropna(subset=['房产名称'])

In [9]:
price_table_jd

,标的物名称,城市名称,行政区名称,小区名称,房屋坐落,房屋面积,所在楼层,总楼层,朝向,是否有电梯,...,产权证号,规划用途,price_date,建成时间,装修程度,房屋性质,登记机关,bedrooms,livingrooms,restrooms
0,海淀区永定路乙1号院9号楼8层902,北京,海淀区,海淀区永定路乙1号院9号楼,北京市海淀区海淀区永定路乙1号院9号楼,212.24 平方米,8,10,南,是,...,京房权证海私移字第0085004号,10住宅,2021-01-21,NaN,NaN,NaN,NaN,4,2,3
1,北京市东城区兴化路6号院3号楼15层1503:102.44(㎡),北京,东城区,大龙公寓,北京市东城区兴化路6号院3号楼15层1503,102.44 平方米,15,21,东南,是,...,京（2018）东不动产权第0013104号,住宅,2021-10-13,NaN,NaN,NaN,NaN,2,1,1
2,北京市东城区民主北街26号院2号楼6层20611:47.11(㎡),北京,东城区,城市亮点,北京市东城区民主北街26号院2号楼6层20611,47.11 平方米,6,12,西向,是,...,京房权证崇私字第73003号,住宅,2020-10-27,NaN,NaN,NaN,NaN,1,1,1
3,北京市东城区竹杆胡同3号楼2层1单元201:73.28(㎡),北京,东城区,北竹杆胡同,北京市东城区竹杆胡同3号楼2层1单元201,73.28 平方米,2,11,南,是,...,X京房权证东字第071717号,住宅,2021-08-26,NaN,NaN,NaN,NaN,2,1,1
4,北京市丰台区丰葆路98号院二区4号楼-2至3层101号\n:427.16(㎡),北京,丰台区,中海九号公馆2区,北京市丰台区丰葆路98号院二区4号楼-2至3层101号,427.16 平方米,,5,南北通透,是,...,X京房权证丰字第451831号,住宅,2021-04-01,NaN,NaN,NaN,NaN,7,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,石景山区海特花园44号楼5层505号:262.62(㎡),北京,石景山区,海特花园,石景山区海特花园44号楼5层505号,262.62 平方米,5,5,东北,否,...,京(2020)石不动产权第0000045号,住宅,2022-01-29,NaN,NaN,NaN,NaN,3,1,2
352,北京市东城区广渠南水关胡同5号楼7层乙单元703:70.94(㎡),北京,东城区,南水关胡同,北京市东城区广渠南水关胡同5号楼7层乙单元703,70.94 平方米,7,16,南向,否,...,京（2016）东城区不动产权第0024985号,住宅,2020-07-13,NaN,NaN,NaN,NaN,2,1,1
353,北京市大兴区半壁店绿茵花园别墅如茵区11号1至2层全部\n:234.3(㎡),北京,大兴区,绿茵花园,北京市大兴区半壁店绿茵花园别墅如茵区11号1至2层全部,234.3 平方米,1,2,南北通透,否,...,X京房权证兴字第161861号,住宅,2021-01-04,NaN,NaN,NaN,NaN,4,1,3
354,北京市大兴区庞各庄镇隆盛西园7号楼1层2-101:82.74(㎡),北京,大兴区,北京市大兴区庞各庄镇隆盛西园,北京市大兴区庞各庄镇隆盛西园7号楼1层2-101,82.74 平方米,1,6,南北通透,否,...,京房权证兴私字第083007号,住宅,2021-02-02,NaN,NaN,NaN,NaN,2,1,1


In [164]:
price_table_ali

,房产名称,房屋类型,房产坐落,所在小区,建筑面积,户型,朝向,所在楼层,建成年代,规划用途,单位面积价格,财产参考总价,本次网络询价时点为
0,北京市海淀区安宁华庭二区9号楼10层3单元1003:108.47(㎡),住宅用房,北京市海淀区安宁华庭二区9号楼10层3单元1003,北京市海淀区安宁华庭二区9号楼10层3单元1003,108.47㎡,2室1厅2卫,南北,10,,住宅,89006.00元/㎡,9654481.00元,2021年11月30日
1,海淀区永定路乙1号院9号楼8层902,住宅用房,北京市海淀区海淀区永定路乙1号院9号楼,海淀区永定路乙1号院9号楼,212.24㎡,4室2厅3卫,南向,8,,10住宅,68163.00元/㎡,14466915.00元,2021年01月21日
2,北京市海淀区西翠路17号院10号楼3层1单元301,住宅用房,北京市海淀区西翠路17号院10号楼3层1单元301,西翠路17号院,256.62㎡,4室2厅2卫,南北,3,,住宅,141917.00元/㎡,36418741.00元,2021年12月02日
3,北京市东城区交道口北二条35号院1号楼3层5门302号:55.13(㎡),住宅用房,北京市东城区交道口北二条35号院1号楼3层5门302号,北京市东城区交道口北二条35号院,55.13㎡,1室1厅1卫,南北,3,,住宅,103915.00元/㎡,5728834.00元,2021年01月19日
5,北京市东城区竹杆胡同3号楼2层1单元201:73.28(㎡),住宅用房,北京市东城区竹杆胡同3号楼2层1单元201,北竹杆胡同,73.28㎡,2室1厅1卫,南向,2,,住宅,103480.00元/㎡,7583014.00元,2021年08月26日
...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,北京市朝阳区朝阳公园路9号院2号楼7单元501号:393.89(㎡),住宅用房,北京市朝阳区朝阳公园路9号院2号楼7单元501号,朝阳公园路九号公寓,393.89㎡,3室2厅4卫,东向,4,,住宅,70182.00元/㎡,27643988.00元,2021年08月17日
342,北京市海淀区上庄三嘉信苑14号楼3层3单元301:66.68(㎡),住宅用房,北京市海淀区上庄三嘉信苑14号楼3层3单元301,上庄三嘉信苑,66.68㎡,2室1厅1卫,南北,3,,住宅,50884.00元/㎡,3392945.00元,2021年10月14日
343,顺义区双阳东区2号楼3层2单元302:88.43(㎡),住宅用房,顺义区双阳东区2号楼3层2单元302,双阳东区,88.43㎡,2室1厅1卫,南北,3,,住宅,29317.00元/㎡,2592502.00元,2021年07月27日
344,顺义区金穗路2号院8号楼8层822:40.78(㎡),住宅用房,顺义区金穗路2号院8号楼8层822,顺义区金穗路2号院,40.78㎡,2室0厅1卫,西向,8,,办公,25152.00元/㎡,1025699.00元,2020年12月28日


# failed file

In [43]:
filename = failed_jd_files[6]
filename

'Foreclosure_Reports\\【二拍】北京市门头沟区惠民家园房产一套\\京东.pdf'

In [36]:
pdf_content=pdfplumber.open(filename)
panel_price=pdf_content.pages[1+1].extract_table()
panel_price

panel_price=pd.DataFrame(np.array(panel_price)[:,1],index=np.array(panel_price)[:,0]).T
list_a=pdf_content.pages[2+1].extract_text().split('\n')
price_date=list(filter(lambda x: '本次网络询价时间为' in x,list_a))[0].split('：')[1]
panel_price['price_date']=price_date
price_table_jd=pd.concat([price_table_jd,panel_price],axis='rows')
        


In [44]:
price_table_jd['price_date']=pd.to_datetime(price_table_jd['price_date'],format='%Y年%m月%d日')
price_table_jd[['bedrooms','livingrooms','restrooms']]=price_table_jd['户型'].str.split('[室厅卫]', expand=True)[[0,1,2]]
price_table_jd=price_table_jd.reset_index(drop=True)    

# store data

In [50]:
price_table_jd.to_parquet('price_table_jd.parquet')

In [51]:
price_table_ali.to_parquet('price_table_ali.parquet')

# draft by teacher

In [ ]:
pdf_content=pdfplumber.open(file_list_ali[12])
pdf_content.pages[1].extract_text()

'2、询价对象所在小区市场成交案例\n成交时间 建筑面积(平方米) 成交总价(元) 成交单价(元) 屋室 朝向 楼层\n2020年04月29日 159.6 9860000 61779 2 南 -/11\n2020年04月10日 122.17 9700000 79397 0 南北 中/0\n3、询价对象所在城市、行政区及小区半年内的市场价格走势\n（二）房产周边情况\n1. 周边配套详情\n北京四中璞瑅学校：南三环东路17号院(紫芳园六区)7号楼；\n十八中附属小学：芳星园二区9号；\n学校 黄城根小学分校(南三环东路辅路)：紫芳园六区7号；\n北京市第十八中学(国际部)：方庄芳星园二区11号楼；\n北京市第十八中学：方庄芳星园2区11号楼；\n地铁站 方庄(地铁站)：14号线东段；\n中国邮政储蓄银行24小时自助银行(紫芳园支行)：方庄路东150米；\n中国邮政储蓄银行(紫芳园支行)：紫芳园四区3号楼1/2层118；\n银行 中国工商银行(北京东方庄支行)：芳城东里9号楼商业用房首层；\n交通银行24小时自助银行(东方庄支行)：芳城东里9-1号；\n中国农业银行(北京紫芳园支行)：紫芳园三区2号楼1层；\n北京开发·红黄蓝双语幼儿园：方庄东路紫芳园二区7号楼(近方庄路)；\n红黄蓝方庄国际幼儿园：方庄镇紫芳园二区七号楼；\n幼儿园 艺术幼儿园：丰台街道方庄镇方庄镇方庄桥东紫芳园六区2号楼；\n巨人学校方庄紫芳园校区：紫芳园3区1号；\n巨人学校小巨人幼教中心(方庄路)：方庄镇紫芳园三区1号；\n方庄东路(公交站)：511路;专161路;专162路;少年宫专线；\n紫芳路(公交站)：511路;专161路;专162路;少年宫专线；\n方庄东路南口(公交站)：专161路;专162路；\n公交车站 方庄桥北(公交站)：139路;352路;684路；\n方庄桥东(公交站)：25路;300路内环;300路外环;300路快内;300路快外;511\n路;53路;652路;680路;687路;848路;973路;985路;986路;夜30路内;特8路内\n环;特8路外；\n同仁堂：方庄路18号附近紫芳园三区4号楼；\n悦康送百姓平安大药房(方庄药店)：芳城东里7号；\n药店 思派大药房北京四季堂店：饮马井路45号；\n四季堂大药房(辅路店)：方庄桥东紫芳园六区5号楼；\n四季堂大药

In [ ]:
info_list

['2、询价对象所在小区市场成交案例',
 '成交时间 建筑面积(平方米) 成交总价(元) 成交单价(元) 屋室 朝向 楼层',
 '2020年04月29日 159.6 9860000 61779 2 南 -/11',
 '2020年04月10日 122.17 9700000 79397 0 南北 中/0',
 '3、询价对象所在城市、行政区及小区半年内的市场价格走势',
 '（二）房产周边情况',
 '1. 周边配套详情',
 '北京四中璞瑅学校：南三环东路17号院(紫芳园六区)7号楼；',
 '十八中附属小学：芳星园二区9号；',
 '学校 黄城根小学分校(南三环东路辅路)：紫芳园六区7号；',
 '北京市第十八中学(国际部)：方庄芳星园二区11号楼；',
 '北京市第十八中学：方庄芳星园2区11号楼；',
 '地铁站 方庄(地铁站)：14号线东段；',
 '中国邮政储蓄银行24小时自助银行(紫芳园支行)：方庄路东150米；',
 '中国邮政储蓄银行(紫芳园支行)：紫芳园四区3号楼1/2层118；',
 '银行 中国工商银行(北京东方庄支行)：芳城东里9号楼商业用房首层；',
 '交通银行24小时自助银行(东方庄支行)：芳城东里9-1号；',
 '中国农业银行(北京紫芳园支行)：紫芳园三区2号楼1层；',
 '北京开发·红黄蓝双语幼儿园：方庄东路紫芳园二区7号楼(近方庄路)；',
 '红黄蓝方庄国际幼儿园：方庄镇紫芳园二区七号楼；',
 '幼儿园 艺术幼儿园：丰台街道方庄镇方庄镇方庄桥东紫芳园六区2号楼；',
 '巨人学校方庄紫芳园校区：紫芳园3区1号；',
 '巨人学校小巨人幼教中心(方庄路)：方庄镇紫芳园三区1号；',
 '方庄东路(公交站)：511路;专161路;专162路;少年宫专线；',
 '紫芳路(公交站)：511路;专161路;专162路;少年宫专线；',
 '方庄东路南口(公交站)：专161路;专162路；',
 '公交车站 方庄桥北(公交站)：139路;352路;684路；',
 '方庄桥东(公交站)：25路;300路内环;300路外环;300路快内;300路快外;511',
 '路;53路;652路;680路;687路;848路;973路;985路;986路;夜30路内;特8路内',
 '环;特8路外；',
 '同仁堂：方庄路18号附近紫芳园

In [151]:
info_list=pdf_content.pages[1].extract_text().split('\n')

In [153]:
add_list=[]
i=0
while i < len(info_list):
    if '询价对象所在小区市场成交案例' in info_list[i]:
        begin_i=i+1
    else:
        pass
    if '询价对象所在城市' in info_list[i]:
        end_i=i
    else:
        pass
    i+=1

In [160]:
pd.DataFrame(info_list[begin_i:end_i])[0].str.split(' ',expand=True)

,0,1,2,3,4,5,6
0,成交时间,建筑面积(平方米),成交总价(元),成交单价(元),屋室,朝向,楼层
1,2020年04月29日,159.6,9860000,61779,2,南,-/11
2,2020年04月10日,122.17,9700000,79397,0,南北,中/0
